In [1]:
%config Completer.use_jedi = False

In [2]:
MI_FOLD = 0

MI_STAGE = 0

GPU_ID = 1 #0, 1, or 2 for balancing resources

NAMING_STRING = f'fold_{MI_FOLD}_stage_{MI_STAGE}'

In [3]:
import numpy as np
import pandas as pd

import glob, os

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
from sklearn.experimental import enable_halving_search_cv

from sklearn.model_selection import StratifiedKFold, HalvingGridSearchCV
from sklearn.preprocessing import StandardScaler

from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

In [5]:
%%time
if MI_STAGE < 2:
    MI_STAGE_LETTER = 'A'
elif MI_STAGE < 4:
    MI_STAGE_LETTER = 'B'
elif MI_STAGE < 6:
    MI_STAGE_LETTER = 'C'
    
    
df = pd.read_csv(f'../data/train_{MI_STAGE_LETTER}_fold_{MI_FOLD}.csv', index_col=0)
test_df = pd.read_csv(f'../data/test_{MI_STAGE_LETTER}_fold_{MI_FOLD}.csv', index_col=0)

/opt/conda/lib/python3.8/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


CPU times: user 1min 42s, sys: 6.6 s, total: 1min 49s
Wall time: 15min


In [10]:
presentation_vars = [col for col in list(pd.read_csv('../presentation.txt', header=None)[0]) if 'ND' not in col]
cath_lab_vars = list(pd.read_csv('../cath_lab.txt', header=None)[0])
med_vars = list(pd.read_csv('../meds.txt', header=None)[0])
pci_vars = list(pd.read_csv('../pci.txt', header=None)[0])
access_vars = [col for col in df.columns if 
               'femor' in col.lower() or
               'radia' in col.lower()]
closure_vars = [col for col in df.columns if 'clos' in col.lower()]
outcome_var = ['Bleed']

all_vars = outcome_var + presentation_vars + access_vars + cath_lab_vars + med_vars + pci_vars + closure_vars

In [12]:
if MI_STAGE == 0:
    stage = presentation_vars
elif MI_STAGE == 1:
    stage = presentation_vars + access_vars
elif MI_STAGE == 2:
    stage = presentation_vars + access_vars + cath_lab_vars
elif MI_STAGE == 3:
    stage = presentation_vars + access_vars + cath_lab_vars + med_vars
elif MI_STAGE == 4:
    stage = presentation_vars + access_vars + cath_lab_vars + med_vars + pci_vars
elif MI_STAGE == 5:
    stage = presentation_vars + access_vars + cath_lab_vars + med_vars + pci_vars + closure_vars

In [13]:
def correct_non_continuous(df):
    _df = df.copy()
    
    #These are variables from the initial df that were binary, and have now undergone Imputation.  This imputation
    #algorithm only provides floats, so the initial part of this function sets those imputed values to whichever
    #is closer, 0 or 1.  It then changes the entire Series to int.
    binary_vars = [
        'AA_BetaBlockers', 'AA_CaChannel', 'AA_LongActingNitrates', 'AA_OtherAgent', 'AA_Ranolazine', 'AntiAnginalMed',
        'CardioLVSD', 'ChronicLungDisease', 'CurrentDialysis', 'Diabetes', 'Dyslipidemia', 'FamilyHxCAD', 'HispOrig',
        'Hypertension', 'OnsetTimeEst', 'OnsetTimeNA', 'PreProcCKMBNM', 'Prior2weeksHF', 'PriorCABG', 'PriorCardiacArrest',
        'PriorCardioShock', 'PriorCVD', 'PriorHF', 'PriorMI', 'PriorPAD', 'PriorPCI', 'Smoker', 'STEMIFirstNoted',
        'ThromTherapy', 'ValveSurgery', 'CardiacTransplant', 'PeriopEval', 'PatientTransPCI', 'DiagCorAngio',
        'DissectionSeg', 'LeftHeartCath', 'PerfSeg', 'PreviousStent', 'PrevTreatedLesion', 'Thrombus', 'BifurcationLesion',
        'ChronicOcclusion', 'CTO', 'CulpritArtery', 'FFR', 'GuidewireLesion', 'IABP', 'InRestenosis', 'InThrombosis', 'IVUS',
        'MVSupport', 'OtherProcedure', 'PCICardioShock',
    ]
    
    for col in binary_vars:
        if df[col].notna().all():
            #print(col)
            temp = (_df[col] >= 0.5)
            _df[col] = temp.astype(int)
            
    #These are ordinal variables from the initial df.  As above, we now set variables to the nearest legal value, then
    #change the dtype to int.
    ordinal_vars = {
        'LesonComplexty':     [1, 2],
        'StentType':          [1, 2],
        'AnginalClass':       [1, 2, 3, 4, 5],
        'CADPresentation':    [1, 2, 3, 4, 5, 6],
        'PCIStatus':          [1, 2, 3, 4],
        'Prior2weekNYHA':     [1, 2, 3, 4],
        'DCathStatus':        [1, 2, 3, 4],
        'DCathTreatment':     [1, 2, 3, 4, 5],
        'PreProcTIMI':        [1, 2, 3, 4],
        'PreTIMI':            [0, 1, 2, 3],
        'IABPTiming':         [1, 2, 3],
        'LesionGraft':        [0, 1, 2, 3],
        'MVSupportTiming':    [1, 2, 3],
        'NEWSEQ':             [0, 1, 2, 3],
    }
    
    for col, values in ordinal_vars.items():
        if _df[col].notna().all():
            #print(col)
            #Set anything more extreme than allowed values to most extreme allowed
            temp = _df[col].round().astype(int)
            temp[temp<values[0]] = values[0]
            temp[temp>values[-1]] = values[-1]
            _df[col] = temp.astype(int)
            
    return _df
        

In [14]:
%%time
df = correct_non_continuous(df)

CPU times: user 20.9 s, sys: 14min 33s, total: 14min 54s
Wall time: 14min 54s


In [15]:
%%time
test_df = correct_non_continuous(test_df)

CPU times: user 2.54 s, sys: 1.16 s, total: 3.7 s
Wall time: 3.96 s


In [16]:
%%time
X = df[stage]
y = df.Bleed

X_test = test_df[stage]
y_test = test_df.Bleed

CPU times: user 1.71 s, sys: 12 s, total: 13.7 s
Wall time: 13.7 s


In [17]:
clf = XGBClassifier(
    max_depth=2,
    n_estimators=1000,
    learning_rate=0.1,
    gpu_id=GPU_ID,
    tree_method='gpu_hist',
    use_label_encoder=False,
    eval_metric='logloss',
    verbosity=3,
)

In [18]:
clf.fit(X,y);

[22:24:46] DEBUG: ../src/tree/updater_gpu_hist.cu:892: [GPU Hist]: Configure
[22:24:47] DEBUG: ../src/common/device_helpers.cu:38: Running nccl init on: 10.0
[22:24:48] ======== Monitor: SketchContainer ========
[22:24:48] MakeCuts: 0.000272s, 1 calls @ 272us

[22:24:48] Prune: 0.000175s, 1 calls @ 175us

[22:24:48] ScanInput: 0.002496s, 1 calls @ 2496us

[22:24:48] Unique: 0.00011s, 1 calls @ 110us

[22:25:17] ======== Monitor: Learner ========
[22:25:17] Configure: 1e-06s, 1 calls @ 1us

[22:25:17] EvalOneIter: 0.008177s, 1000 calls @ 8177us

[22:25:17] GetGradient: 0.148197s, 1000 calls @ 148197us

[22:25:17] PredictRaw: 1.45752s, 1000 calls @ 1457519us

[22:25:17] UpdateOneIter: 28.3327s, 1000 calls @ 28332742us

[22:25:17] ======== Monitor: GBTree ========
[22:25:17] BoostNewTrees: 28.2362s, 1000 calls @ 28236203us

[22:25:17] CommitModel: 1.0088s, 1000 calls @ 1008795us

[22:25:17] ======== Device 1 Memory Allocations:  ========
[22:25:17] Peak memory usage: 3885MiB
[22:25:17] Nu

In [19]:
%%time
pred_test_proba = clf.predict_proba(X_test)[:,1]

[22:25:46] DEBUG: ../src/tree/updater_gpu_hist.cu:892: [GPU Hist]: Configure
[22:25:46] ======== Device 1 Memory Allocations:  ========
[22:25:46] Peak memory usage: 3885MiB
[22:25:46] Number of allocations: 19040
[22:25:46] ======== Monitor:  ========
[22:25:46] ======== NCCL Statistics========
[22:25:46] AllReduce calls: 0
[22:25:46] AllReduce total MiB communicated: 0
CPU times: user 7.48 s, sys: 80 ms, total: 7.56 s
Wall time: 1.41 s


In [21]:
clf.save_model(f'models/{NAMING_STRING}_model_weights.json')

In [22]:
predicted_df = pd.DataFrame(
    data={
    f'{NAMING_STRING}_predicted': pred_test_proba,
    'actual': y_test,
    },
    index=y_test.index,
)

In [23]:
predicted_df.to_csv(f'probs/{NAMING_STRING}_probabilities.csv')

In [24]:
test_auroc = roc_auc_score(y_test, pred_test_proba)

In [28]:
from sklearn.metrics import average_precision_score

In [29]:
def bs_by_hand(actual, pred):
    return np.sum(np.power(pred-actual,2))/actual.shape[0]

In [30]:
def unc_by_hand(actual, pred):
    return np.mean(actual) * (1-np.mean(actual))

In [31]:
def res_by_hand(actual, pred, nbins=100):
    p = np.array(pred)
    a = np.array(actual)
    n = pred.shape[0]
    
    p_breaks = np.linspace(0,1,nbins+1)
    base_rate = a.mean()
    
    res = 0
    
    for i in range(nbins):
        mask = (p > p_breaks[i]) & (p < p_breaks[i+1])
        bin_size = mask.sum()
        if bin_size==0:
            continue
        bin_rate = a[mask].mean()
        res += bin_size * np.power(bin_rate-base_rate, 2)
    return res/n

In [32]:
def rel_by_hand(actual, pred, nbins=100):
    p = np.array(pred)
    a = np.array(actual)
    n = pred.shape[0]
    
    p_breaks = np.linspace(0,1,nbins+1)
    #base_rate = a.mean()
    
    rel = 0
    
    for i in range(nbins):
        mask = (p > p_breaks[i]) & (p < p_breaks[i+1])
        bin_size = mask.sum()
        if bin_size==0:
            continue
        bin_forcast = p[mask].mean()
        bin_rate = a[mask].mean()
        rel += bin_size * np.power(bin_forcast-bin_rate, 2)
    return rel/n

In [33]:
def naive_bss(actual, pred):
    bs_ref = np.sum(np.power(np.mean(actual)-actual,2))/actual.shape[0]
    return 1 - bs_by_hand(actual, pred)/bs_ref

In [25]:
test_auroc

0.8118521016279407

In [34]:
meta_dat = pd.DataFrame(
    index=[10*MI_FOLD + MI_STAGE]
)
meta_dat['stage'] = MI_STAGE
meta_dat['fold'] = MI_FOLD

meta_dat['auroc'] = test_auroc
meta_dat['auprc']     = average_precision_score(y_test, pred_test_proba)
meta_dat['unc']       = unc_by_hand(y_test, pred_test_proba)
meta_dat['res']       = res_by_hand(y_test, pred_test_proba)
meta_dat['rel']       = rel_by_hand(y_test, pred_test_proba)
meta_dat['bs']        = bs_by_hand(y_test, pred_test_proba)
meta_dat['naive_bss'] = naive_bss(y_test, pred_test_proba)

In [35]:
meta_dat

,stage,fold,auroc,auprc,unc,res,rel,bs,naive_bss
0,0,0,0.811852,0.203046,0.035044,0.003336,0.000258,0.031961,0.087969


In [36]:
meta_dat.to_csv(f'results/{NAMING_STRING}_results.csv')